## Imports

In [3]:
import numpy as np
import pandas as pd
import datetime
import math
pd.options.display.max_columns = 50

In [4]:
# Plots
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
sns.set(color_codes=True)

In [9]:
from uszipcode import SearchEngine

## Functions

In [59]:
def zip_five(df):
    '''
    Function
    --------
    Updates df with a "ZIP_FIVE" column (5 digit zip code)
    
    Parameters
    ----------
    df : Pandas DataFrame with a "ZIP" column
    
    Returns
    -------
    Pandas DataFrame with a ZIP_FIVE column (5 digit zip code)
    '''
    df['ZIP_FIVE'] = df['ZIP'].apply(lambda x: x[:5])
    return df

In [51]:
def find_same_zip(df, zip_code):
    '''
    Function
    --------
    Looks through Database and returns Entries with same zip code
    
    Parameters
    ----------
    df : Pandas DataFrame of Charities
    zip_code : Zip Code to Match on
    
    Returns
    -------
    Pandas DataFrame with Entries matching provided zip code
    '''
    zip_code = str(zip_code)
    
    temp_df = df[df['ZIP_FIVE'] == zip_code]
    temp_df_two = df[df['ZIP'] == zip_code]
    temp_df = pd.concat((temp_df,temp_df_two))
    
    return temp_df

In [12]:
def find_major_category(df,category:str):
    '''
    Function
    --------
    Filter a Pandas DataFrame by an NTEE Major Category
    
    Parameters
    ----------
    df : Pandas DataFrame with 'NTEE_Major_Category' column
    category : NTEE Category to Filter
    
    Return
    ------
    Filtered Pandas DataFrame
    '''
    temp_df = df[df['NTEE_Major_Category']==category]
    
    return temp_df

In [13]:
def search_by_zip_and_major_category(df, zipcode, category):
    '''
    Function
    --------
    Filter a Pandas DataFrame by Zip Code and NTEE Major Category
    
    Parameters
    ----------
    df : Pandas DataFrame with 'NTEE_Major_Category' column
    zipcode : Zipcode to Filter on
    category : NTEE Major Category to Filter on
    
    Return
    ------
    Filtered Pandas DataFrame
    '''
    
    temp_df = find_same_zip(df,zipcode)
    temp_df = find_major_category(temp_df,category)
    
    return temp_df

In [14]:
def find_minor_category(df,category:str):
    '''
    Function
    --------
    Filter a Pandas DataFrame by an NTEE Major Category
    
    Parameters
    ----------
    df : Pandas DataFrame with 'NTEE_Major_Category' column
    category : NTEE Category to Filter
    
    Return
    ------
    Filtered Pandas DataFrame
    '''
    temp_df = df[df['NTEE_Minor_Category']==category]
    
    return temp_df

In [15]:
def search_by_zip_and_minor_category(df, zipcode, category):
    '''
    Function
    --------
    Filter a Pandas DataFrame by zipcode and NTEE Minor Category
    
    Parameters
    ----------
    df : Pandas DataFrame with 'NTEE_Major_Category' column
    zipcode : Zipcode to Filter on
    category : NTEE Minor Category to Filter
    
    Return
    ------
    Filtered Pandas DataFrame    
    '''
    temp_df = find_same_zip(df,zipcode)
    temp_df = find_minor_category(temp_df,category)
    
    return temp_df

In [94]:
def find_zip_state(df,zipcode):
    '''
    Function
    --------
    Looks through DataFrame and returns Entries with same state
    
    Parameters
    ----------
    df : Pandas DataFrame of Charities
    zip_code : Zip Code to Match on
    
    Returns
    -------
    Pandas DataFrame with Entries matching provided state
    '''
    search = SearchEngine(simple_zipcode=True)
    state = search.by_zipcode(zipcode).state
    temp_df = df[df['STATE']==state]
    
    return temp_df

In [96]:
def search_zip_to_state_major_category(df, zipcode, category):
    '''
    Function
    --------
    Filter a Pandas DataFrame by Zip Code and NTEE Major Category
    
    Parameters
    ----------
    df : Pandas DataFrame with 'NTEE_Major_Category' column
    zipcode : Zipcode to Filter on
    category : NTEE Major Category to Filter on
    
    Return
    ------
    Filtered Pandas DataFrame
    '''
    temp_df = find_zip_state(df,zipcode)
    temp_df = find_major_category(temp_df,category)
    
    return temp_df

## Loading Data

In [68]:
charity_df = pd.read_csv('All_Regions_w_Category_County_df')

In [69]:
charity_df.drop(columns=['Unnamed: 0'],inplace=True)

In [70]:
zip_five(charity_df);

In [71]:
charity_df.head()

,EIN,NAME,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,FOUNDATION,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,ZIP_FIVE,NTEE_Major_Category,NTEE_Minor_Category,County
0,587764,IGLESIA BETHESDA INC,157 ANDOVER ST,LOWELL,MA,01852-2348,0,3,3,7000,200401,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X21,01852,"Religion-Related, Spiritual Development",Protestant,Middlesex County
1,635913,MINISTERIO APOSTOLICO JESUCRISTO ES EL SENOR INC,454 ESSEX ST,LAWRENCE,MA,01840-1242,0,3,3,7000,200401,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X21,01840,"Religion-Related, Spiritual Development",Protestant,Essex County
2,765634,MERCY CHAPEL INTERNATIONAL,75 MORTON VILLAGE DR APT 408,MATTAPAN,MA,02126-2433,0,3,3,7000,200404,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X20,02126,"Religion-Related, Spiritual Development",Christian,Suffolk County
3,841363,AGAPE HOUSE OF PRAYER,39 GOODALE RD STE 2,MATTAPAN,MA,02126-1527,0,3,3,1700,200412,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X20,02126,"Religion-Related, Spiritual Development",Christian,Suffolk County
4,852649,BETHANY PRESBYTERIAN CHURCH,32 HARVARD ST,BROOKLINE,MA,02445-7994,0,3,3,7000,200504,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X20,02445,"Religion-Related, Spiritual Development",Christian,Norfolk County


In [72]:
rating_df = pd.read_csv('../data/API_DF_CN.csv')

In [73]:
rating_df.head(1)

,charityName,ein,mission,tagLine,categoryID,categoryName,causeID,causeName,rating,assetAmount,classification,deductibility,deductibilityCode,incomeAmount,nteeClassification,nteeType,city,postalCode,stateOrProvince,streetAddress1
0,Florida Breast Cancer Foundation,10694045,Florida Breast Cancer Foundation strives to en...,To end the suffering caused by breast cancer,5,Health,13,"Diseases, Disorders, and Disciplines",4.0,2221944.0,Charitable Organization,Contributions are deductible,1.0,1662411.0,Fund Raising and/or Fund Distribution,Medical Research,North Miami,33181,FL,11900 Biscayne Boulevard


In [74]:
ein_and_rating_df = rating_df[['ein','rating']]

In [76]:
charity_df_rated = charity_df.merge(ein_and_rating_df,how='left',left_on='EIN',right_on='ein')

In [77]:
charity_df_rated.drop(columns=['ein'],inplace=True)

In [78]:
charity_df_rated.head(100)

,EIN,NAME,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,FOUNDATION,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,ZIP_FIVE,NTEE_Major_Category,NTEE_Minor_Category,County,rating
0,587764,IGLESIA BETHESDA INC,157 ANDOVER ST,LOWELL,MA,01852-2348,0,3,3,7000,200401,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X21,01852,"Religion-Related, Spiritual Development",Protestant,Middlesex County,NaN
1,635913,MINISTERIO APOSTOLICO JESUCRISTO ES EL SENOR INC,454 ESSEX ST,LAWRENCE,MA,01840-1242,0,3,3,7000,200401,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X21,01840,"Religion-Related, Spiritual Development",Protestant,Essex County,NaN
2,765634,MERCY CHAPEL INTERNATIONAL,75 MORTON VILLAGE DR APT 408,MATTAPAN,MA,02126-2433,0,3,3,7000,200404,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X20,02126,"Religion-Related, Spiritual Development",Christian,Suffolk County,NaN
3,841363,AGAPE HOUSE OF PRAYER,39 GOODALE RD STE 2,MATTAPAN,MA,02126-1527,0,3,3,1700,200412,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X20,02126,"Religion-Related, Spiritual Development",Christian,Suffolk County,NaN
4,852649,BETHANY PRESBYTERIAN CHURCH,32 HARVARD ST,BROOKLINE,MA,02445-7994,0,3,3,7000,200504,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X20,02445,"Religion-Related, Spiritual Development",Christian,Norfolk County,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,10215213,BOWDOIN COLLEGE,3530 COLLEGE STA,BRUNSWICK,ME,04011-8426,0,3,3,2000,193506,11,1,1,201806.0,9,9,1,0,6,2.201155e+09,594153000.0,237784000.0,B420,04011,Education,Undergraduate College (4-year),Cumberland County,NaN
96,10215227,RUMFORD HOSPITAL,420 FRANKLIN ST,RUMFORD,ME,04276-2104,0,3,3,1000,194209,15,1,1,201806.0,8,8,1,0,6,2.698397e+07,45356583.0,43311913.0,E220,04276,Health - General and Rehabilitative,"Hospital, General",Oxford County,NaN
97,10215686,RICE PUBLIC LIBRARY,8 WENTWORTH ST,KITTERY,ME,03904-1756,0,3,3,1000,194903,15,1,1,201806.0,5,5,1,0,6,9.183100e+05,617892.0,617892.0,B71Z,03904,Education,B71Z,York County,NaN
98,10215910,BANGOR HUMANE SOCIETY,693 MOUNT HOPE AVE,BANGOR,ME,04401-5606,0,3,3,4000,197706,15,5,1,201804.0,6,6,1,0,4,3.074535e+06,1454644.0,1245569.0,D200,04401,Animal-Related,Animal Protection and Welfare,Penobscot County,3.0


In [79]:
find_same_zip(charity_df_rated,'01852')

,EIN,NAME,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,FOUNDATION,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,ZIP_FIVE,NTEE_Major_Category,NTEE_Minor_Category,County,rating
0,587764,IGLESIA BETHESDA INC,157 ANDOVER ST,LOWELL,MA,01852-2348,0,3,3,7000,200401,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X21,01852,"Religion-Related, Spiritual Development",Protestant,Middlesex County,NaN
4209,20534984,NEW LIFE CHRISTIAN MINISTRIES,150 MIDDLESEX ST,LOWELL,MA,01852-2115,0,3,3,7000,200307,10,1,1,NaN,0,0,6,0,12,NaN,NaN,NaN,X20,01852,"Religion-Related, Spiritual Development",Christian,Middlesex County,NaN
6351,30494844,COMMON GROUND DEVELOPMENT CORPORATION,155 MERRIMACK ST,LOWELL,MA,01852-1723,0,3,3,1000,200402,16,1,1,201806.0,6,5,1,0,6,4876940.0,697181.0,697181.0,L21,01852,"Housing, Shelter",Public Housing Facilities,Middlesex County,NaN
7028,42104398,GREATER LOWELL FAMILY Y M C A,35 YMCA DR,LOWELL,MA,01852-4005,0,3,3,1000,193404,16,1,1,201812.0,6,6,1,0,12,4619505.0,4137725.0,4130890.0,P270,01852,Human Services - Multipurpose and Other,"Young Men's or Women's Associations (YMCA, YWC...",Middlesex County,NaN
7030,42104401,GIRLS INCORPORATED OF GREATER LOWELL,220 WORTHEN ST,LOWELL,MA,01852-1823,0,3,3,2000,195712,15,1,1,201812.0,6,6,1,0,12,1835859.0,2281566.0,1540476.0,O220,01852,Youth Development,Girls Clubs,Middlesex County,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141844,823881519,TERRACORPS INC,116 JOHN ST SUITE 405,LOWELL,MA,01852-1128,0,3,3,1000,201806,15,1,1,201808.0,4,4,1,0,8,467518.0,368665.0,368665.0,C50,01852,"Environmental Quality, Protection and Beautifi...",Environmental Beautification and Aesthetics,Middlesex County,NaN
146759,831756993,BO DEVELOPMENT ASSOCIATION INC,100 MASSMILLS DR UNIT 318,LOWELL,MA,01852-1238,0,3,3,1000,201812,16,1,1,NaN,0,0,1,0,12,NaN,NaN,NaN,Q33,01852,"International, Foreign Affairs and National Se...",International Relief,Middlesex County,NaN
147486,832197223,DREAM CENTERS FOR THE PERFORMING ARTS INC,227 FAYETTE ST APT 1A,LOWELL,MA,01852-2377,0,3,3,1200,201811,15,1,1,NaN,0,0,2,0,9,NaN,NaN,NaN,A61,01852,"Arts, Culture and Humanities",Performing Arts Centers,Middlesex County,NaN
150079,834538320,BRIDGING INDEPENDENT LIVING TOGETHER INC,9 STATE ST,LOWELL,MA,01852-5014,0,3,3,1000,201905,15,1,1,NaN,0,0,2,0,3,NaN,NaN,NaN,L99,01852,"Housing, Shelter","Housing, Shelter N.E.C.",Middlesex County,NaN


In [82]:
temp_df = search_by_zip_and_major_category(charity_df_rated,
                                 '11030','Education')

In [87]:
temp_df.sort_values(by=['rating','INCOME_AMT'],ascending=False).head(3)

,EIN,NAME,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,FOUNDATION,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,ZIP_FIVE,NTEE_Major_Category,NTEE_Minor_Category,County,rating
22616,112615673,THE MANHASSET SCHOOL COMMUNITY ASSOCIATION INC,PO BOX 33,MANHASSET,NY,11030-0033,0,3,3,1000,198210,15,1,1,201808.0,5,5,1,0,8,864181.0,892914.0,753261.0,B20I,11030,Education,"Elementary, Secondary Education, K - 12",Nassau County,NaN
83996,300032941,ADVENTURES IN LEARNING INC,1845 NORTHERN BLVD,MANHASSET,NY,11030-3526,0,3,3,1000,200203,15,1,1,201808.0,5,5,1,0,8,556273.0,684178.0,632353.0,B90,11030,Education,Educational Services and Schools - Other,Nassau County,NaN
22870,112741980,MANHASSET AFTER SCHOOL EXPERIENCE INC,C/O MUNSEY PARK SCHOOL 1 HUNT LANE,MANHASSET,NY,11030-0000,0,3,3,1000,198712,16,1,1,201806.0,4,5,1,0,6,449513.0,587798.0,587798.0,B900,11030,Education,Educational Services and Schools - Other,Nassau County,NaN


In [210]:
same_df = charity_df_rated.copy()

In [211]:
len(same_df)

1484617

In [212]:
manhasset_ed_df = search_zip_to_state_major_category(same_df,11030,'Education')

In [275]:
flora_df = search_zip_to_state_major_category(same_df,90275,'Education')

In [276]:
flora_df.head()

,EIN,NAME,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,FOUNDATION,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,ZIP_FIVE,NTEE_Major_Category,NTEE_Minor_Category,County,rating
500117,10563236,AAUW DEL MAR-LEUCADIA CALIFORNIA BRANCH,PO BOX 543,SOLANA BEACH,CA,92075-0543,0,3,3,1000,200211,16,1,1,201806.0,0,0,2,0,6,0.0,0.0,0.0,B21,92075,Education,"Kindergarten, Preschool, Nursery School, Early...",San Diego County,NaN
500150,10567964,CALIFORNIA TAMIL ACADEMY,14435 C BIG BASIN WAY 179,SARATOGA,CA,95070-6082,0,3,3,2000,200210,16,1,1,201807.0,6,5,1,0,7,1586906.0,931952.0,931952.0,B99,95070,Education,Education N.E.C.,Santa Clara County,NaN
500156,10568840,MINDFUL EDUCATION INC,1000 MEZGER DR,WOODLAND,CA,95776-5196,0,3,3,2000,200204,16,1,1,201612.0,0,0,2,0,12,0.0,0.0,0.0,B90,95776,Education,Educational Services and Schools - Other,Yolo County,NaN
500172,10571455,DAGUPENOS CHARITABLE FOUNDATION INTERNATIONAL INC,2261 CAYUGA AVE,SAN FRANCISCO,CA,94112-3901,0,3,3,1000,200304,16,1,1,201812.0,2,3,1,0,12,23682.0,56002.0,17613.0,B82,94112,Education,"Scholarships, Student Financial Aid Services, ...",San Francisco County,NaN
500209,10578548,DON LUGO HIGHSCHOOL SPORTS BOOSTERS CLUB INC,13400 PIPELINE AVE,CHINO,CA,91710-4959,0,3,3,1000,200203,16,1,1,201703.0,2,3,1,0,3,17533.0,99568.0,25564.0,B94,91710,Education,Parent/Teacher Group,San Bernardino County,NaN


In [279]:
add_score_and_sort_df(flora_df,90275,zip_factor = 200000)

,EIN,NAME,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,FOUNDATION,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,ZIP_FIVE,NTEE_Major_Category,NTEE_Minor_Category,County,rating,zip_factor,county_factor,state_factor,score
612689,330709401,AMERICAN BOARD OF CERTIFIED MASTER HAIR,2159 W ROCKINGHORSE RD,RCH PALOS VRD,CA,90275-1603,0,3,3,1200,200910,16,1,1,201901.0,2,4,1,0,1,1.215400e+04,3.990390e+05,3.990390e+05,B30,90275,Education,B30,Los Angeles County,NaN,200000,10,1,7.981259e+10
1104988,330709401,AMERICAN BOARD OF CERTIFIED MASTER HAIR,2159 W ROCKINGHORSE RD,RCH PALOS VRD,CA,90275-1603,0,3,3,1200,200910,16,1,1,201901.0,2,4,1,0,1,1.215400e+04,3.990390e+05,3.990390e+05,B30,90275,Education,B30,Los Angeles County,NaN,200000,10,1,7.981259e+10
1474009,951642394,UNIVERSITY OF SOUTHERN CALIFORNIA,UNIVERSITY PARK,LOS ANGELES,CA,90089-0001,0,3,3,2000,193410,16,1,1,201806.0,9,9,1,0,6,1.219545e+10,5.604030e+09,5.602937e+09,B430,90089,Education,University or Technological Institute,Los Angeles County,NaN,0,10,1,6.724837e+10


In [213]:
len(manhasset_ed_df)

8220

In [251]:
def add_score_and_sort_df(df,zipcode,zip_factor = 20,county_factor = 10,state_factor=1,top_char=3):
    '''
    Function
    --------
    ADD score column to Pandas DataFrame and return top X rated Charities by score
    
    Parameters
    ----------
    df : Pandas DataFrame with 'ZIP_FIVE' column
    zipcode : Zipcode to Filter on
    zip_factor : Weighting Variable for Charities with Same Zip Code
    county_factor : Weighting Variable for Charities with Same Zip Code
    state_factor : Weighting Variable for Charities with Same State
    top_char : Number of "Top" results to return sorted by score descending
    
    Return
    ------
    Filtered Pandas DataFrame
    '''
    county = search.by_zipcode(zipcode).county
    state = search.by_zipcode(zipcode).state
    
    df['zip_factor'] = df['ZIP_FIVE'].apply(lambda x: zip_factor if int(x) == zipcode else 0)
    df['county_factor'] = df['County'].apply(lambda x: county_factor if x == county else 0)
    df['state_factor'] = df['STATE'].apply(lambda x: state_factor if x == state else 0)

    df['score'] = df['INCOME_AMT']*(1+df['zip_factor']+df['county_factor']+df['state_factor'])
    
    temp_df = df.sort_values(by='score',ascending=False).head(top_char)
    
    return temp_df

In [273]:
z = add_score_and_sort_df(manhasset_ed_df,zip_factor=100000,zipcode=11030,top_char=3)

In [274]:
z

,EIN,NAME,STREET,CITY,STATE,ZIP,GROUP,SUBSECTION,AFFILIATION,CLASSIFICATION,RULING,FOUNDATION,ORGANIZATION,STATUS,TAX_PERIOD,ASSET_CD,INCOME_CD,FILING_REQ_CD,PF_FILING_REQ_CD,ACCT_PD,ASSET_AMT,INCOME_AMT,REVENUE_AMT,NTEE_CD,ZIP_FIVE,NTEE_Major_Category,NTEE_Minor_Category,County,rating,zip_factor,county_factor,state_factor,score
22616,112615673,THE MANHASSET SCHOOL COMMUNITY ASSOCIATION INC,PO BOX 33,MANHASSET,NY,11030-0033,0,3,3,1000,198210,15,1,1,201808.0,5,5,1,0,8,864181.0,892914.0,753261.0,B20I,11030,Education,"Elementary, Secondary Education, K - 12",Nassau County,NaN,100000,10,1,8.930211e+10
83996,300032941,ADVENTURES IN LEARNING INC,1845 NORTHERN BLVD,MANHASSET,NY,11030-3526,0,3,3,1000,200203,15,1,1,201808.0,5,5,1,0,8,556273.0,684178.0,632353.0,B90,11030,Education,Educational Services and Schools - Other,Nassau County,NaN,100000,10,1,6.842601e+10
22870,112741980,MANHASSET AFTER SCHOOL EXPERIENCE INC,C/O MUNSEY PARK SCHOOL 1 HUNT LANE,MANHASSET,NY,11030-0000,0,3,3,1000,198712,16,1,1,201806.0,4,5,1,0,6,449513.0,587798.0,587798.0,B900,11030,Education,Educational Services and Schools - Other,Nassau County,NaN,100000,10,1,5.878685e+10


In [257]:
len(charity_df_rated)

1484617

In [290]:
download_df = charity_df_rated.drop(columns=['GROUP','SUBSECTION','AFFILIATION','CLASSIFICATION','RULING','FOUNDATION'])

In [292]:
download_df = download_df[['EIN','NAME','INCOME_CD','ZIP_FIVE','NTEE_Major_Category','NTEE_Minor_Category','County','rating']]

In [294]:
download_df.to_csv('../data/charity_info_categories.csv')